# Simulation of luminosity function

In this notebook we simulate the luminosity function using SkyPy. The Schechter parameters are obtained through a fit to COSMOS data by Will and sent over via Slack in the galaxie_paper channel.

In [1]:
import numpy as np
import skypy
from skypy.pipeline import Pipeline
from astropy.table import Table, vstack
import matplotlib.pyplot as plt
from astropy.io import fits
from astropy.table import Table, Column
import skypy_x

print(skypy.__version__)

0.6.dev17+gb2840ce


In [2]:
#Problem: the SEDs don't extend far enough into uv.

In [3]:
#skypy_x.galaxies_paper.gaussian_error

In [4]:
#import speclite.filters

## Sim whole redshift range

In [5]:
# Parameters
log_phi = np.exp(-5.48997506)
phi_exp = -0.19843827
phi_exp_inv = np.divide(1., phi_exp)
M_star = -21.72410256
M_exp = -0.14627561
alpha = -1.27094795

z_start = 0.01
z_end = 4.5

In [6]:
config = f"""parameters:
  log_phi: {log_phi}
  phi_exp: {phi_exp}
  M_star: {M_star}
  M_exp: {M_exp}
  alpha: {alpha}
  phi_exp-1: {phi_exp_inv}
cosmology: !astropy.cosmology.FlatLambdaCDM
  H0: 70
  Om0: 0.3
mag_lim: 27
sky_area: 1.27 deg2
filters:
  des: [ decam2014-g, decam2014-r, decam2014-i, decam2014-z ]
  hsc: [ hsc2017-g, hsc2017-r, hsc2017-i, hsc2017-z, hsc2017-y ]
bands_des: 'griz'
bands_hsc: 'grizy'
mag_limits:
  des: [23.4, 23.2, 22.5, 21.8]
  hsc: [27.8, 27.4, 27.1, 26.6, 25.6]
z_range: !numpy.geomspace [{z_start}, {z_end}, 100]
tables:
  SF:
    z, M_r: !skypy.galaxies.schechter_lf
      redshift: $z_range
      M_star: !astropy.modeling.models.Linear1D [$M_exp, $M_star]
      phi_star: !astropy.modeling.models.Exponential1D [$log_phi, $phi_exp-1] # astropy function requires 1/tau as second argument
      alpha: $alpha
      m_lim: $mag_lim
      sky_area: $sky_area
    coeff: !skypy.galaxies.spectrum.dirichlet_coefficients
      redshift: !numpy.zeros_like [$SF.z]
      alpha0: [1.171, 3.055, 1.394, 1.669, 1.855]
      alpha1: [2.385, 4.294, 0.898, 1.895, 1.459]
      weight: [3.47e+09, 3.31e+06, 2.13e+09, 1.64e+10, 1.01e+09]
    sm: !skypy.galaxies.spectrum.kcorrect.stellar_mass
      coefficients: $SF.coeff
      magnitudes: $SF.M_r
      filter: hsc2017-r
    sm_remain: !skypy.galaxies.spectrum.kcorrect.stellar_mass_remain
      coefficients: $SF.coeff
      magnitudes: $SF.M_r
      filter: hsc2017-r
    m_des_g, m_des_r, m_des_i, m_des_z: !skypy.galaxies.spectrum.kcorrect.apparent_magnitudes
      redshift: $SF.z
      coefficients: $SF.coeff
      stellar_mass: $SF.sm
      filters: $filters.des
    m_hsc_g, m_hsc_r, m_hsc_i, m_hsc_z, m_hsc_y: !skypy.galaxies.spectrum.kcorrect.apparent_magnitudes
      redshift: $SF.z
      coefficients: $SF.coeff
      stellar_mass: $SF.sm
      filters: $filters.hsc
"""

In [7]:
# f = open("../config_files/cosmos_lf_automated.yml", 'w')
# f.write(config)
# f.close()

In [8]:
import speclite.filters

fdir = 'filters/'
speclite.filters.load_filters(fdir+'vista-Y.ecsv', 
                              fdir+'vista-J.ecsv',
                              fdir+'vista-H.ecsv',
                              fdir+'vista-Ks.ecsv')

In [9]:
# Execute SkyPy simulation
sim = Pipeline.read("config_DES_DF_SF.yaml")
#sim = Pipeline.read("../config_files/cosmos_lf_automated.yml")
sim.execute()


In [10]:
print(len(sim['SF']))
skypy_sf = vstack([sim['SF']])

579120


In [11]:
sim2 = Pipeline.read("config_DES_DF_pass1.yaml")
sim2.execute()
sim3 = Pipeline.read("config_DES_DF_pass2.yaml")
sim3.execute()
skypy_p = vstack([sim2['passive'], sim3['passive']])

In [12]:
skypy_galaxies = vstack([sim['SF'], sim2['passive'], sim3['passive']])

In [13]:
print(type(skypy_galaxies))

<class 'astropy.table.table.Table'>


In [14]:
print(len(skypy_galaxies['z']))
skypy_galaxies

789272


z,M_r,coeff [5],sm,sm_remain,m_des_true [5],m_hsc_true [5],m_vista_true [4],m_des_sim [5],des_sigma [5],m_hsc_sim [5],hsc_sigma [5],m_vista_sim [4],vista_sigma [4]
float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64
0.31531143605496276,-15.09140721338857,0.12650356252473105 .. 0.04184732648697167,174400515.6672137,95186729.62437397,27.260025692571975 .. 25.525355067595324,26.83309707532074 .. 25.478032730123655,25.414002643211386 .. 24.72714088206482,29.18627470324315 .. 25.788987294838257,0.8687837878276311 .. 0.3350660366458229,26.78984680742709 .. 25.476368533305404,0.04476014141355225 .. 0.09705284804060949,25.48172134621223 .. 24.662312346776826,0.25140097721295573 .. 0.266474780615858
1.7254755668545032,-20.434936611271095,0.9170789441126033 .. 0.06473883806954582,22298422031.17235,13650638142.53266,26.634930610846254 .. 25.60086891783977,26.513066672856276 .. 25.487626927700852,25.2997187252425 .. 23.567498866944888,26.909156520656307 .. 25.752306591923904,0.48861311224587206 .. 0.3591800973684295,26.479861968676396 .. 25.62278065186019,0.03341893966249893 .. 0.09791294115470947,25.296784571814083 .. 23.501335811354533,0.22632582576735605 .. 0.0918350056843633
2.7004604292111596,-19.75761427187365,0.00017453053925302737 .. 0.1116909468391465,7154144700.444047,3959927198.6237693,26.73163131548835 .. 26.35680349647076,26.486022456866976 .. 26.323320133180708,26.301816194429684 .. 25.66418049922072,26.532082402045187 .. 26.722574021286302,0.5341061764150703 .. 0.7203127433490379,26.414320423365403 .. 26.66879639364964,0.032605034758558804 .. 0.21123321581850552,26.83719375209632 .. 25.303316213353185,0.5689652445042744 .. 0.6311094074983041
3.123453208340849,-22.76067951165978,0.1261378118531077 .. 0.5498394489107818,31795570726.363186,22079886715.949883,24.90461506221852 .. 23.680404984586755,24.046324155894393 .. 23.701053959928938,23.691049784372236 .. 22.91974972001012,24.89875660684762 .. 23.61230586790804,0.09946184174110072 .. 0.061478085150761294,24.048957006308175 .. 23.69975597094852,0.0037340180809857606 .. 0.01900898582275233,23.667774294885465 .. 22.92238032923817,0.051760192254365935 .. 0.050746046336971044
3.4935711837351335,-21.19638245351332,0.008886860865905536 .. 0.24143187963896912,21855775985.65253,12758391990.4447,29.774029320338006 .. 26.343710682511837,27.313430464697863 .. 26.255459212059517,26.234251401841963 .. 24.81001827504976,28.285433893962118 .. 27.102253379870447,8.798471637459347 .. 0.7116819688458211,27.37575281221162 .. 26.29873545218505,0.06947963822989166 .. 0.19844383901945914,26.906153919164016 .. 24.744034842101993,0.534663422720052 .. 0.28758114752594627
0.8889046376624827,-19.18214212663944,0.09045804078924567 .. 0.5257083191358393,1452288699.79327,992399741.8130955,25.67965261134546 .. 24.094738252702797,25.400363730103017 .. 24.05553092801552,24.045064136454045 .. 23.833639794082153,26.038327029566563 .. 24.06201741671388,0.20283753753860054 .. 0.08991776546674078,25.400949146198467 .. 24.085943879023574,0.012205277986945894 .. 0.026290887113856406,24.104720023228598 .. 23.7387345340288,0.07155217150661695 .. 0.11723706497980853
0.8415053782666496,-20.23204111348992,0.11430014389617299 .. 0.8271562613542325,3263693962.921565,2404198614.9381504,24.36392695117663 .. 22.881748524393473,24.024544105965504 .. 22.853119066347716,22.821686105372457 .. 22.716237477620297,24.17158571673625 .. 22.87749121840529,0.06053983672359297 .. 0.029602479771706647,24.022308808204162 .. 22.847331015467866,0.0036659891742366764 .. 0.008785230745692279,22.80929818455088 .. 22.659107683826743,0.023470125603954912 .. 0.04213851269432234
0.8620649346807326,-17.738178361174334,0.37287064174310386 .. 0.2944949070605896,336256131.67907655,217107681.64241156,25.4323093971379 .. 25.269381920489046,25.4714494631814 .. 25.321824575848638,25.397697266020963 .. 25.424320099241434,25.360031336227784 .. 24.969779376370724,0.16156215804125465 .. 0.264749317

In [15]:
phdu = fits.PrimaryHDU()
phdu.header.append(('Redsift', "z", 'galaxy redshift'))
phdu.header.append(('Magnitude', "M_r", 'galaxys absolute magnitude in HSC r-band')) 
phdu.header.append(('Coeff', "coeff", 'Dirichlet coefficients'))
phdu.header.append(('SM', "sm", 'stellar mass'))
phdu.header.append(('SM_Remain', "sm_remain", 'stellar mass_remain'))
phdu.header.append(('App. Mag.', "m_survey_band", 'hsc: g,r,i,z,y ; des: g,r,i,z ; vista: Y,J,H,Ks'))

hdu = fits.table_to_hdu(skypy_galaxies)
hdu.header['EXTNAME'] = 'Galaxies'

hdul = fits.HDUList([phdu, hdu])
file_path = 'lf_whole_z_range_remain.fits'
hdul.writeto(file_path, overwrite=True)

In [16]:
hdul[0].header

SIMPLE  =                    T / conforms to FITS standard                      
BITPIX  =                    8 / array data type                                
NAXIS   =                    0 / number of array dimensions                     
EXTEND  =                    T                                                  
REDSIFT = 'z       '           / galaxy redshift                                
HIERARCH Magnitude = 'M_r     ' / galaxys absolute magnitude in HSC r-band      
COEFF   = 'coeff   '           / Dirichlet coefficients                         
SM      = 'sm      '           / stellar mass                                   
HIERARCH SM_Remain = 'sm_remain' / stellar mass_remain                          
HIERARCH App. Mag. = 'm_survey_band' / hsc: g,r,i,z,y ; des: g,r,i,z ; vista: Y,

In [17]:
print(len(hdul['Galaxies'].data['M_r']))

789272


In [ ]:
phdu = fits.PrimaryHDU()
phdu.header.append(('Redsift', "z", 'galaxy redshift'))
phdu.header.append(('Magnitude', "M_r", 'galaxys absolute magnitude in HSC r-band')) 
phdu.header.append(('Coeff', "coeff", 'Dirichlet coefficients'))
phdu.header.append(('SM', "sm", 'stellar mass'))
phdu.header.append(('SM_Remain', "sm_remain", 'stellar mass_remain'))
phdu.header.append(('App. Mag.', "m_survey_band", 'hsc: g,r,i,z,y ; des: g,r,i,z ; vista: Y,J,H,Ks'))

hdu = fits.table_to_hdu(skypy_sf)
hdu.header['EXTNAME'] = 'Galaxies'

hdul = fits.HDUList([phdu, hdu])
file_path = 'lf_whole_z_range_remain_sf.fits'
hdul.writeto(file_path, overwrite=True)

In [ ]:
phdu = fits.PrimaryHDU()
phdu.header.append(('Redsift', "z", 'galaxy redshift'))
phdu.header.append(('Magnitude', "M_r", 'galaxys absolute magnitude in HSC r-band')) 
phdu.header.append(('Coeff', "coeff", 'Dirichlet coefficients'))
phdu.header.append(('SM', "sm", 'stellar mass'))
phdu.header.append(('SM_Remain', "sm_remain", 'stellar mass_remain'))
phdu.header.append(('App. Mag.', "m_survey_band", 'hsc: g,r,i,z,y ; des: g,r,i,z ; vista: Y,J,H,Ks'))

hdu = fits.table_to_hdu(skypy_p)
hdu.header['EXTNAME'] = 'Galaxies'

hdul = fits.HDUList([phdu, hdu])
file_path = 'lf_whole_z_range_remain_p.fits'
hdul.writeto(file_path, overwrite=True)

In [ ]:
print(stop_here)

## Sim in bins

Since SkyPy does only allow `alpha` being a scalar, we simulate catlogues in different redshift bins.

In [ ]:
num_bins = 5
redshift_bin = np.linspace(z_start, z_end, num_bins)
print(redshift_bin)
bin_centers = (redshift_bin[:-1] + redshift_bin[1:])/2
print(bin_centers)

In [ ]:
# Parameters
log_phi = np.exp(-5.48997506)
phi_exp = -0.19843827
phi_exp_inv = np.divide(1., phi_exp)
M_star = -21.72410256
M_exp = -0.14627561

def alpha_func(z): 
    return 0.07071859 * z + -1.27094795

In [ ]:
def write_config(filename, alpha, z_min, z_max):
    config = f"""parameters:
  log_phi: {log_phi}
  phi_exp: {phi_exp}
  M_star: {M_star}
  M_exp: {M_exp}
  alpha: {alpha}
  phi_exp-1: {phi_exp_inv}
cosmology: !astropy.cosmology.FlatLambdaCDM
  H0: 70
  Om0: 0.3
mag_lim: 30
sky_area: 1.27 deg2
filters:
  des: [ decam2014-g, decam2014-r, decam2014-i, decam2014-z ]
  hsc: [ hsc2017-g, hsc2017-r, hsc2017-i, hsc2017-z, hsc2017-y ]
bands_des: 'griz'
bands_hsc: 'grizy'
mag_limits:
  des: [23.4, 23.2, 22.5, 21.8]
  hsc: [27.8, 27.4, 27.1, 26.6, 25.6]
z_range: !numpy.geomspace [{z_min}, {z_max}, 100]
tables:
  SF:
    z, M_r: !skypy.galaxies.schechter_lf
      redshift: $z_range
      M_star: !astropy.modeling.models.Linear1D [$M_exp, $M_star]
      phi_star: !astropy.modeling.models.Exponential1D [$log_phi, $phi_exp-1] # astropy function requires 1/tau as second argument
      alpha: $alpha
      m_lim: $mag_lim
      sky_area: $sky_area
    coeff: !skypy.galaxies.spectrum.dirichlet_coefficients
      redshift: !numpy.zeros_like [$SF.z]
      alpha0: [1.171, 3.055, 1.394, 1.669, 1.855]
      alpha1: [2.385, 4.294, 0.898, 1.895, 1.459]
      weight: [3.47e+09, 3.31e+06, 2.13e+09, 1.64e+10, 1.01e+09]
    sm: !skypy.galaxies.spectrum.kcorrect.stellar_mass_remain
      coefficients: $SF.coeff
      magnitudes: $SF.M_r
      filter: hsc2017-r
    m_des_g, m_des_r, m_des_i, m_des_z: !skypy.galaxies.spectrum.kcorrect.apparent_magnitudes
      redshift: $SF.z
      coefficients: $SF.coeff
      stellar_mass: $SF.sm
      filters: $filters.des
    m_hsc_g, m_hsc_r, m_hsc_i, m_hsc_z, m_hsc_y: !skypy.galaxies.spectrum.kcorrect.apparent_magnitudes
      redshift: $SF.z
      coefficients: $SF.coeff
      stellar_mass: $SF.sm
      filters: $filters.hsc
"""
    f = open(filename, 'w')
    f.write(config)
    f.close()

In [ ]:
phdu = fits.PrimaryHDU()
phdu.header.append(('Redsift', "z", 'galaxy redshift'))
phdu.header.append(('Magnitude', "M_r", 'galaxys absolute magnitude in HSC r-band')) 
phdu.header.append(('Coeff', "coeff", 'Dirichlet coefficients'))
phdu.header.append(('SM', "sm", 'stellar mass'))
phdu.header.append(('App. Mag.', "m_survey_band", 'hsc: g,r,i,z,y ; des: g,r,i,z'))

hdu_list = [phdu]

for i, z in enumerate(bin_centers):
    alpha = alpha_func(z)
    filename = "../config_files/cosmos_lf_automated.yml"
    write_config(filename, alpha, redshift_bin[i], redshift_bin[i+1])
    sim = Pipeline.read(filename)
    sim.execute()
    galaxies = vstack([sim['SF']])
    hdu = fits.table_to_hdu(galaxies)
    hdu.header['EXTNAME'] = str(z)
    hdu_list.append(hdu)
    
hdul = fits.HDUList(hdu_list)
file_path = 'lf_z_bin.fits'
hdul.writeto(file_path, overwrite=True)

In [ ]:
length = 0
for i, z in enumerate(bin_centers):
    a = len(hdul[str(z)].data['z'])
    print(a)
    length += a
    
print('total: ', length)